In [ ]:
import yfinance as yf
from bs4 import BeautifulSoup as BS
import requests
from textblob import TextBlob
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re
import tkinter as tk
from tkinter import ttk, messagebox
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams

# Function to get the ticker symbol from company name using Gemini API
def get_ticker_symbol(company_name, api_key):
    url = f"https://api.gemini.com/v1/symbols"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from Gemini API. Status code: {response.status_code}")
        return None
    
    symbols = response.json()
    ticker_symbol = None
    
    for symbol in symbols:
        if company_name.lower() in symbol.lower():
            ticker_symbol = symbol
            break
    
    return ticker_symbol

# Function to get financial data using yfinance
def get_financial_data(company_name, api_key):
    ticker_symbol = get_ticker_symbol(company_name, api_key)
    if ticker_symbol:
        stock = yf.Ticker(ticker_symbol)
        stock_info = stock.info
        financial_data = {
            "Ticker Symbol": ticker_symbol,
            "Current Price": stock_info.get('currentPrice', 'N/A'),
            "Market Cap": stock_info.get('marketCap', 'N/A'),
            "P/E Ratio": stock_info.get('trailingPE', 'N/A'),
            "P/B Ratio": stock_info.get('priceToBook', 'N/A'),
            "Dividend Yield": stock_info.get('dividendYield', 'N/A'),
            "Debt-to-Equity Ratio": stock_info.get('debtToEquity', 'N/A'),
            "Return on Equity (ROE)": stock_info.get('returnOnEquity', 'N/A'),
            "Revenue": stock_info.get('totalRevenue', 'N/A'),
            "Gross Profit": stock_info.get('grossProfits', 'N/A'),
            "Net Income": stock_info.get('netIncomeToCommon', 'N/A')
        }
        return financial_data
    else:
        return None

# Function to extract news articles related to the company
def get_news(company_name):
    url = f"https://www.businesstoday.in/topic/{company_name}"
    webpage = requests.get(url)
    
    if webpage.status_code != 200:
        return []
    
    trav = BS(webpage.content, "html.parser")
    news_list = []
    for link in trav.find_all('a'):
        if link.string and len(link.string) > 35:
            news_list.append(link.string.strip())
    
    return news_list

# Sentiment analysis on news articles
def perform_sentiment_analysis(news_list):
    sentiment_scores = []
    for news in news_list:
        blob = TextBlob(news)
        sentiment_scores.append(blob.sentiment.polarity)
    return sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0

# Extract financial data from PDF
def extract_text_from_pdf(pdf_path):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                extracted_text += text + "\n"
            else:
                images = convert_from_path(pdf_path, first_page=page.page_number, last_page=page.page_number)
                for img in images:
                    extracted_text += pytesseract.image_to_string(img) + "\n"
    return extracted_text

# Function to process financial data from the PDF text
def extract_all_data(text):
    patterns = {
        'Revenue': r'Revenues?\s+totaled\s+\$?([\d,.]+)\s+billion',
        'Net Income': r'Net income attributable to.*?\$?([\d,.]+)\s+billion',
        'EPS': r'\$([\d.]+)\s+per\s+diluted\s+share',
    }
    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            data[key] = match.group(1)
    return data

# GUI Setup
def display_financial_data():
    company_name = company_entry.get()
    if company_name:
        financial_data = get_financial_data(company_name, api_key)
        if financial_data:
            result_text.delete(1.0, tk.END)
            for key, value in financial_data.items():
                result_text.insert(tk.END, f"{key}: {value}\n")
        else:
            messagebox.showerror("Error", "Could not fetch financial data for the company.")
    else:
        messagebox.showerror("Error", "Please enter a company name.")

def display_news_and_sentiment():
    company_name = company_entry.get()
    if company_name:
        news_list = get_news(company_name)
        if news_list:
            sentiment = perform_sentiment_analysis(news_list)
            result_text.delete(1.0, tk.END)
            for news in news_list:
                result_text.insert(tk.END, f"{news}\n\n")
            result_text.insert(tk.END, f"Average Sentiment: {sentiment}\n")
        else:
            messagebox.showerror("Error", "No news found for the company.")
    else:
        messagebox.showerror("Error", "Please enter a company name.")

def process_pdf():
    pdf_path = pdf_entry.get()
    if pdf_path:
        text = extract_text_from_pdf(pdf_path)
        data = extract_all_data(text)
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, f"Extracted Data: {data}\n")
    else:
        messagebox.showerror("Error", "Please provide a valid PDF path.")

# Create the main window
root = tk.Tk()
root.title("Financial and PDF Data Processor")

# Company name input
company_label = tk.Label(root, text="Enter Company Name:")
company_label.pack(pady=5)
company_entry = tk.Entry(root, width=50)
company_entry.pack(pady=5)

# Buttons for actions
fetch_data_btn = tk.Button(root, text="Get Financial Data", command=display_financial_data)
fetch_data_btn.pack(pady=5)

fetch_news_btn = tk.Button(root, text="Get News & Sentiment", command=display_news_and_sentiment)
fetch_news_btn.pack(pady=5)

# PDF file input
pdf_label = tk.Label(root, text="Enter PDF Path:")
pdf_label.pack(pady=5)
pdf_entry = tk.Entry(root, width=50)
pdf_entry.pack(pady=5)

# Button to process PDF
process_pdf_btn = tk.Button(root, text="Process PDF", command=process_pdf)
process_pdf_btn.pack(pady=5)

result_text = tk.Text(root, height=15, width=80)
result_text.pack(pady=5)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_650905/2202601506.py", line 117, in display_financial_data
    financial_data = get_financial_data(company_name, api_key)
NameError: name 'api_key' is not defined
